<a href="https://colab.research.google.com/github/mamatqobulovsardor/Aviachipta_narxlarini_bashorati/blob/main/Aviachipta_narxlarini_bashorat_qilish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'avia-sales:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5854619%2F9597431%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T014103Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1464af7d6c23ee14e2ced66f45623ba8820a8f906607a02f3b9e0c36c2534534d88da058cd665f4404eab37b34f3e5339cad32450b4380cbf8d182b1415c47f7f1653329ac1b707239a4030afc276fe965606811cd3ea3daec999461e339033aef20a99357094c74ff368f68440c45135cf6bc81f25eb9d06e2afb34039fa5c834db1ebe661b45ba05a5c07b6e8a14848df932b5336f2e59a902ddabcfc108a37c63dadd6ac5ce64227b5343d1ff14d095deba2d20023d54c20885b02dca25e1d50f92d04bb66179615ecdbf0c689a69250beb16da6e03a0350fb868b65cb5ca37b046b756d6bce3528ecec7597beae287b6e46bd61ae659765e72e00c51b700'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 379346 bytes downloaded
Downloaded and uncompressed: avia-sales
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, __,filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

/kaggle/input/avia-sales/train_data.csv
/kaggle/input/avia-sales/test_data.csv


In [ ]:
train_data = pd.read_csv("/content/train_data.csv")
test_data = pd.read_csv("/content/test_data.csv")
solution_data = pd.read_csv("/content/sample_solution.csv")

In [ ]:
train_data.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [ ]:
#train_data.info()

In [ ]:
#train_data.isna()

In [ ]:
train_data.sum()

,0
id,200010000
airline,VistaraSpiceJetVistaraVistaraIndigoAir_IndiaVi...
flight,UK-810SG-5094UK-846UK-7066E-5394AI-636UK-834AI...
source_city,BangaloreHyderabadBangaloreKolkataChennaiDelhi...
departure_time,Early_MorningEveningMorningMorningEarly_Mornin...
stops,onezerooneonezerooneonezerooneoneoneonezeroone...
arrival_time,NightNightEveningEveningMorningMorningMorningA...
destination_city,MumbaiKolkataDelhiHyderabadMumbaiChennaiMumbai...
class,EconomyEconomyBusinessEconomyEconomyBusinessEc...
duration,243552.53


In [ ]:
train_data.isna().sum()

,0
id,0
airline,0
flight,0
source_city,0
departure_time,0
stops,0
arrival_time,0
destination_city,0
class,0
duration,0


In [ ]:
train_data.describe()

,id,duration,days_left,price
count,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,12.177627,25.924150,20960.281700
std,5773.647028,7.157944,13.624874,22775.459535
min,1.000000,0.830000,1.000000,1105.000000
25%,5000.750000,6.830000,14.000000,4783.000000
50%,10000.500000,11.250000,26.000000,7425.000000
75%,15000.250000,16.080000,38.000000,42521.000000
max,20000.000000,38.580000,49.000000,114523.000000


In [ ]:
x = train_data.drop("price", axis=1)
y = train_data["price"]

In [ ]:
x.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4


In [ ]:
y.head()

,price
0,7212
1,5292
2,60553
3,5760
4,10712


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

nums = ['duration', 'days_left']
cats = ['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']

num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, nums),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cats)
])

train = full_pipeline.fit_transform(x)

In [ ]:
train.toarray()

array([[ 0.28952803, -0.36141789,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.45682723, -1.38897606,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.36291029, -1.53577008,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 3.23458153, -0.65500594,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.58012154, -0.36141789,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.84836171,  0.00556716,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
random_forest_m = RandomForestRegressor()

In [ ]:
random_forest_m.fit(train,y)

RandomForestRegressor()

In [ ]:
RF_predict = random_forest_m.predict(full_pipeline.fit_transform(x))

from sklearn.metrics import mean_absolute_error
Natija = mean_absolute_error(y, RF_predict)
Natija

568.0928891208333